# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Для начала импортируем необходимые для работы инструменты.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Теперь прочитаем датафреймы.

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')                   
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Посмотрим общую информацию о таблицах

In [3]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Мы видим, что пропусков в таблицах нет, везде одинаковое число строк, но различается тип данных. 
**id** — уникальный идентификатор скважины;
**f0, f1, f2** — три признака точек (неважно, что они означают, но сами признаки значимы);
**product** — объём запасов в скважине (тыс. баррелей).

Выведем на экран таблицы для наглядности.

In [6]:
display(df_0)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [7]:
display(df_1)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [8]:
display(df_2)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


Во всех таблицах мы видим как положительные, так и отрицательные значения. Проверим данные на наличие дубликатов.

In [9]:
df_0.duplicated().sum()

0

In [10]:
df_1.duplicated().sum()

0

In [11]:
df_2.duplicated().sum()

0

Дубликаты во всех таблицах отсутствуют.

Изучим корреляцию по каждому датафрейму.

In [12]:
df_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Мы наблюдаем различную корреляцию по столбцам - она низкая или средняя.

In [13]:
df_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Мы наблюдаем различную корреляцию по столбцам - она низкая или средняя, а также заметна высокая коореляция между столбцами product и f2.

In [14]:
df_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Мы наблюдаем различную корреляцию по столбцам - она низкая или средняя.

## Обучение и проверка модели

Сначала необходимо разделить выборки на обучающие и валидационные по каждому датафрейму.

In [15]:
data0_train, data0_valid = train_test_split(df_0, test_size=0.25, random_state=12345)
data1_train, data1_valid = train_test_split(df_1, test_size=0.25, random_state=12345)
data2_train, data2_valid = train_test_split(df_2, test_size=0.25, random_state=12345)


features_train0 = data0_train.drop(['id', 'product'], axis=1)
target_train0 = data0_train['product']
features_valid0 = data0_valid.drop(['id', 'product'], axis=1)
target_valid0 = data0_valid['product']

features_train1 = data1_train.drop(['id', 'product'], axis=1)
target_train1 = data1_train['product']
features_valid1 = data1_valid.drop(['id', 'product'], axis=1)
target_valid1 = data1_valid['product']

features_train2 = data2_train.drop(['id', 'product'], axis=1)
target_train2 = data2_train['product']
features_valid2 = data2_valid.drop(['id', 'product'], axis=1)
target_valid2 = data2_valid['product']

Оценим размер полученных выборок.

In [16]:
print('Обучающая выборка первого датафрейма:',features_train0.shape)
print(target_train0.shape)
print('Валидационная выборка первого датафрейма:',features_valid0.shape)
print(target_valid0.shape)
print()
print('Обучающая выборка второго датафрейма:',features_train1.shape)
print(target_train1.shape)
print('Валидационная выборка второго датафрейма:',features_valid1.shape)
print(target_valid1.shape)
print()
print('Обучающая выборка третьего датафрейма:',features_train2.shape)
print(target_train2.shape)
print('Валидационная выборка третьего датафрейма:',features_valid2.shape)
print(target_valid2.shape)

Обучающая выборка первого датафрейма: (75000, 3)
(75000,)
Валидационная выборка первого датафрейма: (25000, 3)
(25000,)

Обучающая выборка второго датафрейма: (75000, 3)
(75000,)
Валидационная выборка второго датафрейма: (25000, 3)
(25000,)

Обучающая выборка третьего датафрейма: (75000, 3)
(75000,)
Валидационная выборка третьего датафрейма: (25000, 3)
(25000,)


Обучим молель с помощью линейной регрессии и расчитаем показатели первой таблицы.

In [17]:
model = LinearRegression()
model.fit(features_train0,target_train0) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid0) # предсказания модели на валидационной выборке
data0_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid0,predictions_valid)**0.5
data0_predictions_valid = model.predict(features_valid0)

print("Средний запас предсказанного сырья в месторождении", data0_pred_mean)
print("Средний фактический запас сырья в месторождении", target_valid0.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result)

Средний запас предсказанного сырья в месторождении 92.59256778438035
Средний фактический запас сырья в месторождении 92.07859674082927
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


Обучим модель с помощью линейной регрессии и расчитаем показатели второй таблицы.

In [18]:
model = LinearRegression()
model.fit(features_train1,target_train1) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid1) # предсказания модели на валидационной выборке
data1_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid1,predictions_valid)**0.5
data1_predictions_valid = model.predict(features_valid1)

print("Средний запас предсказанного сырья в месторождении", data1_pred_mean)
print("Средний фактический запас сырья в месторождении", target_valid1.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result)

Средний запас предсказанного сырья в месторождении 68.728546895446
Средний фактический запас сырья в месторождении 68.72313602435997
RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


Обучим молель с помощью линейной регрессии и расчитаем показатели третьей таблицы.

In [19]:
model = LinearRegression()
model.fit(features_train2,target_train2) # обучение модели на тренировочной выборке
predictions_valid = model.predict(features_valid2) # предсказания модели на валидационной выборке
data2_pred_mean=predictions_valid.mean()
result = mean_squared_error(target_valid2,predictions_valid)**0.5
data2_predictions_valid = model.predict(features_valid2)

print("Средний запас предсказанного сырья в месторождении", data2_pred_mean)
print("Средний фактический запас сырья в месторождении", target_valid2.mean())
print("RMSE модели линейной регрессии на валидационной выборке:", result)

Средний запас предсказанного сырья в месторождении 94.96504596800489
Средний фактический запас сырья в месторождении 94.88423280885438
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


Мы видим, что самый высокий средний запас предсказанного сырья, а также RMSE в третьем датафрейме. Самое низкое значение RMSE во второй таблице, оно равно 0.89.

## Подготовка к расчёту прибыли

Посчитаем сколько денег из бюджета выделяют на одну из 200, которые будут отобраны.

In [27]:
MONEY_ONE_WELL = 10000000000 / 200
print(MONEY_ONE_WELL)

50000000.0


А также посчитаем сколько нужно тысяч баррелей, чтобы окупить затраты.

In [28]:
COUNT_BARR = MONEY_ONE_WELL / 450000
print(COUNT_BARR)

111.11111111111111


Получили значение в 111 тысяч баррелей, чтобы окупить затраты.

Расчитаем объём сырья для безубыточной разработки новой скважины по каждой таблице, средние значения мы уже рассчитывали в предыдущем шаге, продублируем значения для удобства.

In [29]:
df_0_mean=92.07859674082927
df_1_mean=68.72313602435997
df_2_mean=94.88423280885438

print('Объём сырья для безубыточной разработки новой скважины 1 региона:', COUNT_BARR - df_0_mean)
print('Объём сырья для безубыточной разработки новой скважины 2 региона:', COUNT_BARR - df_1_mean)
print('Объём сырья для безубыточной разработки новой скважины 3 региона:', COUNT_BARR - df_2_mean)

Объём сырья для безубыточной разработки новой скважины 1 региона: 19.032514370281845
Объём сырья для безубыточной разработки новой скважины 2 региона: 42.38797508675114
Объём сырья для безубыточной разработки новой скважины 3 региона: 16.226878302256736


Самый большой объём сырья для безубыточной разработки новой скважины мы видим во втором датафрейме, разница между первой и третьей таблицей небольшая.

## Расчёт прибыли и рисков 

Перейдём к расчёту прибыли и рисков. Для начала напишем собственную функцию для расчёта прибыльности 200 лучших скважин.

In [34]:
def profit(target, predictions, count):
    profit = 0
    probs_sorted = predictions.sort_values(ascending=False).head(200)
    selected = target[probs_sorted.index][:count] 
    for top in selected:
        profit += (top - COUNT_BARR)*450000
    return profit

Перейдём к непосредственному расчету прибыльности.

In [35]:
data0_valid['product_pred']=data0_predictions_valid

profit0=profit(data0_valid['product'], data0_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit0/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 3321 млн.


/tmp/ipykernel_31/3372057489.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data0_valid['product_pred']=data0_predictions_valid


In [36]:
data1_valid['product_pred']=data1_predictions_valid

profit1=profit(data1_valid['product'], data1_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit1/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 2415 млн.


/tmp/ipykernel_31/2564123118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_valid['product_pred']=data1_predictions_valid


In [37]:
data2_valid['product_pred'] = data2_predictions_valid

profit2=profit(data2_valid['product'], data2_valid['product_pred'],200)
print("Прибиыльность 200 топовых скважин валидационной выборки", round(profit2/1000000), "млн.")

Прибиыльность 200 топовых скважин валидационной выборки 2710 млн.


/tmp/ipykernel_31/3913746591.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2_valid['product_pred'] = data2_predictions_valid


Самую высокую прибыльность показывают скважины из первой таблицы.

Теперь рассчитаем вероятные убытки.

In [38]:
def boots (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    counter=0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        preds_subsample = predictions[target_subsample.index]
        
        values.append(profit(target_subsample, preds_subsample, 500))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    higher = values.quantile(0.975)
    
    print('Вероятность убытков',stats.percentileofscore(values, 0),'%')
    print("Среднее значение бутстрепа", (values.mean())/1000000, "млн.")
    print("Верхняя граница доверительного интервала", higher)
    print("Нижняя граница доверительного интервала", lower)
    print()
print("Для первого региона")
boots(data0_valid['product'], data0_valid['product_pred'])
print()
print("Для второго региона")
boots(data1_valid['product'], data1_valid['product_pred'])
print()
print("Для третьего региона")
boots(data2_valid['product'], data2_valid['product_pred'])

Для первого региона
Вероятность убытков 7.6 %
Среднее значение бутстрепа 403.535244261165 млн.
Верхняя граница доверительного интервала 931574367.5508461
Нижняя граница доверительного интервала -118327362.12311177


Для второго региона
Вероятность убытков 1.7 %
Среднее значение бутстрепа 465.04105822107704 млн.
Верхняя граница доверительного интервала 864388463.3254703
Нижняя граница доверительного интервала 32036742.850023583


Для третьего региона
Вероятность убытков 7.9 %
Среднее значение бутстрепа 414.20972284096746 млн.
Верхняя граница доверительного интервала 972339167.9202834
Нижняя граница доверительного интервала -148373898.68249217



Вероятность убытков самая низкая в регионе из второй таблицы - 1,7%, разница вероятности убытков между первой и третьей таблицей невелика. Стоит выбирать между скважинами из первой и второй таблицы, так как в первом случае довольно высокая прибыльность и даже с учетом возможных убытков она остаётся таковой, а во втором случае низкая вероятность убытков.

## Вывод

Итак, в ходе выполнения проекта мы вополнили преобработку данных, обучили модель линейной регрессии, предсказали средний запас сырья в месторождении, рассчитали RMSE модели, а также прибыль и возможные риски по каждому региону.

Самый высокий средний запас предсказанного сырья, а также RMSE в третьем датафрейме. Самое низкое значение RMSE во второй таблице, оно равно 0.89.

Исходя из наших рассчётов следует выбирать между скважинами из первого и второго региона, так как в первом случае довольно высокая прибыльность и даже с учетом возможных убытков она остаётся таковой, а во втором случае низкая вероятность убытков. Я бы рекомендовала выбрать второй регион, так как и RMSE модели там ниже.

Необходимо добыть и продать 111,1 тысяч баррелей, чтобы окупить затраты.